In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from track1 import *
backbone = generate_backbone(bpm,beat_per_bar,bar_per_emphasis, n_bars_buildup, progression)

handling {'part': 'intro', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'no'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'no'), ('melody1', 'yes'), ('melody2', 'no')]}
handling {'part': 'pause', 'buildup': 1.0, 'duration': 4, 'tracks': [('base', 'yes'), ('chords', 'no'), ('drums1', 'no'), ('drums2', 'no'), ('melody1', 'no'), ('melody2', 'no')]}
handling {'part': 'verse', 'buildup': 1.0, 'duration': 12, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 0.5, 'duration': 6, 'tracks': [('base', 'yes'), ('chords', 'yes'), ('drums1', 'yes'), ('drums2', 'yes'), ('melody1', 'yes'), ('melody2', 'yes')]}
handling {'part': 'verse', 'buildup': 0.0, 'duration': 6, 'tracks': [('base', 'yes'), ('ch

In [17]:
from magenta.music.midi_io import sequence_proto_to_midi_file, sequence_proto_to_pretty_midi
import pickle

with open("pickled_track1_drums", "rb") as f:
    full_drum_seq=pickle.load(file=f)

# Control change

In [18]:
for cc in [93,91,75,5]:
    chorus_target = 128

    start_chorus = 0
    drums = full_drum_seq.instruments
    for b in backbone:
        buildup = b["buildup_factor"]

        start_time = b["start_time"]
        end_time = b["end_time"]
        #
        # Chorus
        #
        if buildup > 0:
            end_chorus = math.floor(buildup**2 * chorus_target)

            for chorus in range(start_chorus, end_chorus):

                rat = (chorus - start_chorus)/(end_chorus - start_chorus)

                time = start_time * (1-rat) + end_time * rat

                for drum in drums:
                    drum.control_changes.append(pretty_midi.ControlChange(cc, chorus, time))

            start_chorus = end_chorus+1
        elif start_chorus > 0:
            for drum in drums:
                drum.control_changes.append(pretty_midi.ControlChange(cc, 0, start_time))
            start_chorus = 0

# PitchBend

In [19]:
pitch_target = 8191

start_pitch = 0
drums = full_drum_seq.instruments

for drum in drums:
    drum.control_changes.append(pretty_midi.ControlChange(100, 0, 0))
    drum.control_changes.append(pretty_midi.ControlChange(101, 0, 0))
    drum.control_changes.append(pretty_midi.ControlChange(6, 64, 1))
    
for b in backbone:
    buildup = b["buildup_factor"]
    
    start_time = b["start_time"]
    end_time = b["end_time"]
    #
    # Pitch
    #
    if buildup > 0:
        end_pitch = math.floor(buildup**2 * pitch_target)

        for pitch in range(start_pitch, end_pitch):

            rat = (pitch - start_pitch)/(end_pitch - start_pitch)

            time = start_time * (1-rat) + end_time * rat

            for drum in drums:
                drum.pitch_bends.append(pretty_midi.PitchBend(pitch, time))
            
        start_pitch = end_pitch+1
    elif start_pitch > 0:
        for drum in drums:
            drum.pitch_bends.append(pretty_midi.PitchBend(0, start_time))
        start_pitch = 0

    

In [20]:
generate_mp3(full_drum_seq.fluidsynth())

In [55]:
import pandas as pd
pd.DataFrame(backbone)

,base,beat_per_bar,buildup_factor,chords,current_bar,drums1,drums2,duration,end_time,final_bar,melody1,melody2,noise_buildup,part,start_time
0,0.0,8,0.000,0.0,0,0.0,0.0,4,3.692308,False,1.0,0.0,True,intro,0.000000
1,0.0,8,0.250,0.0,1,0.0,0.0,4,7.384615,False,1.0,0.0,True,intro,3.692308
2,0.0,8,0.500,0.0,2,0.0,0.0,4,11.076923,False,1.0,0.0,True,intro,7.384615
3,0.0,8,0.750,0.0,3,0.0,0.0,4,14.769231,True,1.0,0.0,True,intro,11.076923
4,1.0,8,0.000,1.0,4,1.0,0.0,12,18.461538,False,1.0,0.0,True,verse,14.769231
5,1.0,8,0.000,1.0,5,1.0,0.0,12,22.153846,False,1.0,0.0,True,verse,18.461538
6,1.0,8,0.000,1.0,6,1.0,0.0,12,25.846154,False,1.0,0.0,True,verse,22.153846
7,1.0,8,0.000,1.0,7,1.0,0.0,12,29.538462,False,1.0,0.0,True,verse,25.846154
8,1.0,8,0.000,1.0,8,1.0,0.0,12,33.230769,False,1.0,0.0,True,verse,29.538462
9,1.0,8,0.125,1.0,9,1.0,0.0,12,36.923077,False,1.0,0.0,True,verse,33.230769
